# Running bash commands from your notebook

First, let's install all the dependencies. 

You can directly run bash commands in your notebook, by either prefixing your commands with an exclamation mark `!`:
```ipython
[1] !echo "this is a bash command"
this is a bash command

[2] !ls
/home/user/git_repos/FNO_workshop
```

or by starting your cell with the `%%bash` ipython magic. 

Let's see a simple example:

In [1]:
%%bash

for var in hello world
do
  echo ${var} 
done

hello
world


# Installing the dependencies

Now, let's install the dependencies.

```bash
%%bash 

target_folder='./temp'
[ -d foo ] || mkdir -p ${target_folder}

git clone https://github.com/tensorly/tensorly ${target_folder}/tensorly
cd ${target_folder}/tensorly
pip install -e .

git clone https://github.com/tensorly/torch ${target_folder}/torch
cd ${target_folder}/torch
pip install -e .

git clone https://github.com/NeuralOperator/neuraloperator ${target_folder}/neuraloperator
cd ${target_folder}/neuraloperator
pip install -e .
```

In [ ]:
!pip install gpustat
!pip install gdown
!pip install h5py wandb ruamel.yaml

# Test you have everything installed

In [2]:
import tensorly as tl
import tltorch
import neuralop as no

print(f'{tl.__version__=}')
print(f'{tltorch.__version__=}')
print(f'{no.__version__=}')

tl.__version__='0.8.0'
tltorch.__version__='0.3.0'
no.__version__='0.1.0'
